# Scenario: Classic RCT

We call 'Classic Randomized Controlled Trial' (RCT) a scenario where a treatment is randomly assigned to participants, and we do not have pre-experiment data of participants like pre-treatment outcome.

Treatment - new onboarding for new users.

We will test hypothesis:

$H_o$ - There is no difference in conversion rate between treatment and control groups.

$H_a$ - There is a difference in conversion rate between treatment and control groups.

In [1]:
import numpy as np
from causalis.data.dgps import generate_classic_rct_26
from causalis.data import CausalData

We will use gold dgp from causalis library. More you can read at

In [2]:
data = generate_classic_rct_26(return_causal_data=False)
data.head()

,conversion,d,platform_ios,country_usa,source_paid
0,0.0,0.0,1.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0
3,0.0,1.0,1.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0


In [3]:
causaldata = CausalData(df = data,
                        treatment='d',
                        outcome='conversion',
                        confounders=['platform_ios', 'country_usa', 'source_paid'])

In [4]:
from causalis.statistics.functions import outcome_stats
outcome_stats(causaldata)

,count,mean,std,min,p10,p25,median,p75,p90,max
treatment,,,,,,,,,,
0.0,4955,0.198991,0.399281,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1.0,5045,0.232904,0.422723,0.0,0.0,0.0,0.0,0.0,1.0,1.0


# Monitoring

In [5]:
from causalis.scenarios.rct import check_srm

check_srm(assignments=causaldata, target_allocation={0: 0.5, 1: 0.5}, alpha=0.001)

SRMResult(status=no SRM, p_value=0.36812, chi2=0.8100, df=1)

# Check the confounders balance

In [6]:
from causalis.statistics.functions import confounders_balance

confounders_balance(causaldata)

,mean_d_0,mean_d_1,abs_diff,smd,ks_pvalue
confounders,,,,,
source_paid,0.299092,0.313776,0.014684,0.031853,0.64592
platform_ios,0.494046,0.502874,0.008828,0.017654,0.98861
country_usa,0.586276,0.591873,0.005597,0.011374,1.00000


# Estimation with Diff-in-Means

In [7]:
from causalis.statistics.models.diff_in_means import DiffInMeans

model = DiffInMeans().fit(causaldata)

### What is `conversion_z_test`

The `conversion_z_test` performs a statistical comparison of conversion rates between two groups (Treatment and Control). It provides a p-value for the hypothesis test, and robust confidence intervals for both absolute and relative differences.

#### 1. Observed Metrics
For each group (Control $0$, Treatment $1$):
- $n_0, n_1$: Total number of observations.
- $x_0, x_1$: Number of successes (conversions).
- $p_0 = \frac{x_0}{n_0}, \;\; p_1 = \frac{x_1}{n_1}$: Observed conversion rates.

#### 2. Hypothesis Test (P-value)
The test evaluates $H_0: p_1 = p_0$ (no difference).
- **Pooled Proportion**: $\hat{p} = \frac{x_0 + x_1}{n_0 + n_1}$
- **Pooled Standard Error**: $SE_{pooled} = \sqrt{\hat{p}(1 - \hat{p}) \left(\frac{1}{n_0} + \frac{1}{n_1}\right)}$
- **Z-Statistic**: $Z = \frac{p_1 - p_0}{SE_{pooled}}$
- **P-value**: $2 \times (1 - \Phi(|Z|))$, where $\Phi$ is the standard normal CDF.

#### 3. Absolute Difference (Newcombe CI)
To calculate the confidence interval for the difference $\Delta = p_1 - p_0$, we use the **Newcombe** method, which is more robust than standard Wald intervals for conversion rates.
1.  **Wilson Score Interval** for each group:
    $$CI_{Wilson, i} = (l_i, u_i) = \frac{p_i + \frac{z^2}{2n_i} \pm z \sqrt{\frac{p_i(1 - p_i)}{n_i} + \frac{z^2}{4n_i^2}}}{1 + \frac{z^2}{n_i}}$$
2.  **Combined Interval**:
    $$CI_{\Delta} = (l_1 - u_0, \;\; u_1 - l_0)$$
    *(where $z$ is the critical value for the chosen $\alpha$)*

#### 4. Relative Difference (Lift)
Lift measures the percentage change: $\text{Lift} = (\frac{p_1}{p_0} - 1) \times 100\%$.
The confidence interval is calculated in the **log-Relative Risk (RR)** scale to handle uncertainty in the denominator:
- **Log-RR Standard Error**: $SE_{\ln(RR)} = \sqrt{\frac{1}{x_1} - \frac{1}{n_1} + \frac{1}{x_0} - \frac{1}{n_0}}$
- **Relative CI**: $(\exp[\ln(\frac{p_1}{p_0}) \pm z \times SE_{\ln(RR)}] - 1) \times 100\%$
*(Small constants are added if $x=0$ using the Haldane–Anscombe correction).*

In [8]:
result_conversion = model.estimate('conversion_ztest')
result_conversion.summary()

,estimand,coefficient,p_val,lower_ci,upper_ci,relative_diff_%,is_significant
0,ATE,0.033913,0.000038,0.011108,0.056658,17.04246,True


In [9]:
result_conversion

CausalEstimate(estimand='ATE', model='DiffInMeans', model_options={'method': 'conversion_ztest', 'alpha': 0.05}, value=0.03391294694870284, ci_upper_absolute=0.05665834451230578, ci_lower_absolute=0.011107630056575168, value_relative=17.04245964815645, ci_upper_relative=26.161257674372564, ci_lower_relative=8.582758392024402, alpha=0.05, p_value=3.794497600839719e-05, is_significant=True, n_treated=5045, n_control=4955, outcome='conversion', treatment='d', confounders=['platform_ios', 'country_usa', 'source_paid'], instrument=[], time=datetime.datetime(2026, 1, 9, 17, 54, 35, 312018), diagnostic_data=DiagnosticData(), sensitivity_analysis={})